In [ ]:
from aqchem.chemistry import Solute, elements
from aqchem.equilibria import Equilibrium, EqSystemLog, EqSystemLin, composition_balance
from IPython.display import Latex, display
import periodictable
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def show(s): display(Latex('$'+s+'$'))

In [ ]:
substances = Hp, OHm, NH4p, NH3, H2O, Cupp, CuNH31pp, CuNH32pp, CuNH33pp, CuNH34pp, CuNH35pp, Cu2OH2pp, CuOH3m, CuOH4mm, CuOH2 = [#, CuOHp, CuOH2, =
    Solute(n, latex_name=l, formula=periodictable.formula(n)) for n, l in [
        ('H{+}', 'H^+'), ('HO{-}', 'OH^-'), ('NH3 + H{+}', 'NH_4^+'),
        ('NH3', 'NH_3'), ('H2O', 'H_2O'), ('Cu{2+}', 'Cu^{2+}'), ('Cu{2+}NH3', 'Cu(NH_3)^{2+}'),
        ('Cu{2+}(NH3)2', 'Cu(NH_3)_2^{2+}'), ('Cu{2+}(NH3)3', 'Cu(NH_3)_3^{2+}'),
        ('Cu{2+}(NH3)4', 'Cu(NH_3)_4^{2+}'), ('Cu{2+}(NH3)5', 'Cu(NH_3)_5^{2+}'), 
        ('2Cu{2+} + 2HO{-}', 'Cu_2(OH)_2^{2+}'),
        ('Cu{2+} + 3HO{-}', 'Cu(OH)_3^-'), ('Cu{2+} + 4HO{-}', 'Cu(OH)_4^{2-}'),
        ('Cu{2+} + 2HO{-}', 'Cu(OH_2)(s)'),
    ]]
CuOH2.solid = True

In [ ]:
show(', '.join([s.latex_name for s in substances])) # + r'(\mathrm{%s})' % (str(s.formula.atoms) + ', ' + str(s.charge))

In [ ]:
init_conc = {Hp: 1e-7, OHm: 1e-7, NH4p: 0, NH3: 1.0, Cupp: 1e-2, 
            CuNH31pp: 0, CuNH32pp: 0, CuNH33pp: 0, CuNH34pp: 0, CuNH35pp: 0,
            H2O: 55.5, Cu2OH2pp: 0, CuOH2: 0, CuOH3m: 0, CuOH4mm: 0}

In [ ]:
# Data från Jämviktskonstanttabell, Avdelningen för oorganisk kemi 2007
H2O_c = init_conc[H2O]
w_autop = Equilibrium({H2O: 1}, {Hp: 1, OHm: 1}, 10**-14/H2O_c)
NH4p_pr = Equilibrium({NH4p: 1}, {Hp: 1, NH3: 1}, 10**-9.26)
CuOH2_s = Equilibrium({CuOH2: 1}, {Cupp: 1, OHm: 2}, 10**-18.8)
CuOH_B3 = Equilibrium({CuOH2: 1, OHm: 1}, {CuOH3m: 1}, 10**-3.6)
CuOH_B4 = Equilibrium({CuOH2: 1, OHm: 2}, {CuOH4mm: 1}, 10**-2.7)
Cu2OH2 = Equilibrium({Cupp: 2, H2O: 2}, {Cu2OH2pp: 1, Hp: 2}, 10**-10.6 / H2O_c**2)
CuNH3_B1 = Equilibrium({CuNH31pp: 1}, {Cupp: 1, NH3: 1}, 10**-4.3)
CuNH3_B2 = Equilibrium({CuNH32pp: 1}, {Cupp: 1, NH3: 2}, 10**-7.9)
CuNH3_B3 = Equilibrium({CuNH33pp: 1}, {Cupp: 1, NH3: 3}, 10**-10.8)
CuNH3_B4 = Equilibrium({CuNH34pp: 1}, {Cupp: 1, NH3: 4}, 10**-13.0)
CuNH3_B5 = Equilibrium({CuNH35pp: 1}, {Cupp: 1, NH3: 5}, 10**-12.4)
equilibria = w_autop, NH4p_pr, CuNH3_B1, CuNH3_B2, CuNH3_B3, CuNH3_B4, CuNH3_B5, Cu2OH2, CuOH_B3, CuOH_B4, CuOH2_s
skip_subs, skip_eq = (4, 4) # (0, 0), (1, 1), (3, 3), (4, 4), (11, 9)
skip_subs, skip_eq = (3, 3)
simpl_subs = substances[:-skip_subs]
simpl_eq = equilibria[:-skip_eq]
simpl_c0 = {k: init_conc[k] for k in simpl_subs}

In [ ]:
show(', '.join([s.latex_name for s in simpl_subs]))
show('~')
from math import log10
for eq in simpl_eq:
    ltx = eq.latex()
    show(ltx + '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~lgK = {0:12.5g}'.format(log10(eq.params)))
[(idx, s.name, init_conc[s]) for idx, s in enumerate(simpl_subs)]

In [ ]:
import sympy as sp
sp.init_printing()
lin_eqsys = EqSystemLin(simpl_eq, simpl_subs)
x, i = sp.symarray('x', lin_eqsys.ns), sp.symarray('i', lin_eqsys.ns)
lin_eqsys.f(x, i, ln=sp.log, exp=sp.exp)

In [ ]:
eqsys = EqSystemLog(simpl_eq, simpl_subs)
f = eqsys.f(x, i, ln=sp.log, exp=sp.exp) #, rref_equil=True, rref_preserv=True)
f

In [ ]:
sp.Matrix(1, len(f), lambda _, q: f[q]).jacobian(x)

In [ ]:
len(f), eqsys.ns

In [ ]:
#eqsys.root(simpl_c0, method='lm', rref=False, norm=True, pres_norm=True, scaling=1e12, delta=1e3, logC=True)
#eqsys.root({H2O: 55.5, Hp: 1e-10, OHm: 1e-3, NH3: 1.0, NH4p: 1e-3}, delta=1e-6, method='lm')
x, sol = eqsys.root(simpl_c0)
assert sol.success
x

In [ ]:
import numpy as np
plt.figure(figsize=(12,8))
Cout_logC, inits_logC, success = eqsys.solve_and_plot(
    simpl_c0, NH3, np.logspace(-3, 0, 100),
    #roots_kwargs=dict(rref_equil=True, rref_preserv=True)
)
all(success)

In [ ]:
sp = Cout_logC[:, eqsys.as_substance_index(Cupp)]*Cout_logC[:, eqsys.as_substance_index(OHm)]**2
varied = inits_logC[:, eqsys.as_substance_index(NH3)]
plt.loglog(varied, sp, label='[$%s$][$%s$]$^2$' % (Cupp.latex_name, OHm.latex_name))
plt.loglog(varied, Cout_logC[:, eqsys.as_substance_index(Hp)], ls=':', label='[$%s$]' % Hp.latex_name)
plt.loglog([varied[0], varied[-1]], [10**-18.8, 10**-18.8], 'k--', label='$K_{sp}(Cu(OH)_2(s))$')
plt.xlabel('[$NH_3$]')
_ = plt.legend()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20,8))
eqsys.plot_errors(Cout_logC, inits_logC, NH3, axes=axes)

In [ ]:
for eq in equilibria:
    show(eq.latex())

In [ ]:
reaction_system = EqSystem(simple_eq, simpl_subs)

In [ ]:
import sympy as sp
#sp.init_printing()
y = sp.symarray('y', len(substances))
s = sp.symbols('s')

In [ ]:
composition_balance(substances, y, 0)

In [ ]:
jmat = j(np.log(x0) if logC else x0)
plt.imshow(jmat, cmap='binary', interpolation='None')
jmat[:7, :4].tolist()